## Building A Chatbot
In this  We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions



In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")




In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001AC54F35CF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AC54F70100>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is jagadesh chilla and I am a Aspiring AI Engineer")])

AIMessage(content="Hello Jagadesh Chilla, it's nice to meet you! \n\nThat's great to hear you're aspiring to be an AI Engineer. It's a fascinating and rapidly growing field.  \n\nWhat aspects of AI engineering are you most interested in? Do you have any specific areas you're focusing on or projects you're working on?  \n\nI'd love to hear more about your journey and maybe offer some helpful resources or advice along the way.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 27, 'total_tokens': 129, 'completion_time': 0.185454545, 'prompt_time': 0.001410539, 'queue_time': 0.251042041, 'total_time': 0.186865084}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2ee0fd15-2fde-4d8c-b183-9addcb0f02cd-0', usage_metadata={'input_tokens': 27, 'output_tokens': 102, 'total_tokens': 129})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is jagadesh chilla and I am a Aspiring AI Engineer"),
        AIMessage(content="Hello Jagadesh Chilla! It's nice to meet you. \n\nAs a Aspiring AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Jagadesh Chilla, and you are an aspiring AI Engineer! \n\nIs there anything else you'd like to tell me about yourself or your work?  😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 107, 'total_tokens': 148, 'completion_time': 0.074545455, 'prompt_time': 0.002810089, 'queue_time': 0.253151281, 'total_time': 0.077355544}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6cc93981-5942-40b1-9764-31ea11ccae56-0', usage_metadata={'input_tokens': 107, 'output_tokens': 41, 'total_tokens': 148})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [10]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is jagadesh chilla and I am a Aspiring AI Engineer")],
    config=config
)

In [11]:
response.content

"Hi Jagadesh, it's great to meet you!\n\nThat's awesome that you're aspiring to be an AI Engineer! It's a rapidly growing field with so much potential.  \n\nWhat sparked your interest in AI?  Are there any particular areas of AI that you're most drawn to? \n\nI'm happy to answer any questions you have or offer any advice I can based on my knowledge.\n"

In [12]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Jagadesh Chilla. 😊  \n\nHow can I help you today?  Do you have any questions about AI or anything else on your mind?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 212, 'total_tokens': 249, 'completion_time': 0.067272727, 'prompt_time': 0.005749049, 'queue_time': 0.275034474, 'total_time': 0.073021776}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9c332362-1fb9-49ea-986a-ea4fad2bb679-0', usage_metadata={'input_tokens': 212, 'output_tokens': 37, 'total_tokens': 249})

In [14]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. \n\nWhat can I do for you today?  \n\n"

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  I remember. 😊  \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [16]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [18]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is jagadesh")]})

AIMessage(content="Hi Jagadesh, it's nice to meet you!\n\nI'm happy to help answer your questions to the best of my ability.  \n\nWhat can I do for you today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 33, 'total_tokens': 79, 'completion_time': 0.083636364, 'prompt_time': 0.00147641, 'queue_time': 0.25044471, 'total_time': 0.085112774}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a326278f-236e-4e0a-bf86-af5bfee1e690-0', usage_metadata={'input_tokens': 33, 'output_tokens': 46, 'total_tokens': 79})

In [19]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [20]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is jagadesh")],
    config=config
)

response

AIMessage(content="Hello Jagadesh, it's nice to meet you!  \n\nI'm here to help. What can I do for you today? Ask me anything! 😊  I'll do my best to answer your questions. \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 33, 'total_tokens': 84, 'completion_time': 0.092727273, 'prompt_time': 0.00155261, 'queue_time': 0.25146921, 'total_time': 0.094279883}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6be94bfb-9e52-4e01-a4da-2296ffe72825-0', usage_metadata={'input_tokens': 33, 'output_tokens': 51, 'total_tokens': 84})

In [21]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jagadesh.  I remember! 😊  \n\nIs there anything else I can help you with? \n'

In [22]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [23]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is jagadesh")],"language":"Telugu"})
response.content

'నమస్తే జగదేశ్! \n\nనేను మీకు సహాయకరమైన సహాయకుడిని. మీ ప్రశ్నలకు నా శక్తి అనుసారం తెలుగులో సమాధానం ఇస్తే, సంతోషిస్తాను. ఏమి అడిగితే? 😊 \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [24]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [25]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am jagadesh")],"language":"Telugu"},
    config=config
)
repsonse.content

'మీకు స్వాగతం Jagadesh! 😊\n\nఏమి సలహా అవసరం? నేను మీకు సాయుు చేయడానికి సిద్ధంగా ఉన్నాను. \n'

In [26]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Telugu"},
    config=config,
)

In [27]:
response.content

'మీ పేరు Jagadesh అని నేను తెలుసుకున్నాను. 👋 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [28]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

C:\Users\Welcome\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Welcome\OneDrive\Documents\AI\GEN AI\LCEL\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Welcome\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [29]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your taste preferences.  \n\nWhat's your favorite ice cream flavor? 😊🍦\n"

In [30]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2".  😊  \n\n\n\nI\'m always ready for more math problems (or any other questions you have)!\n'

In [31]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [32]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nWould you like to tell me your name? 😊\n"

In [33]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I have no memory of past conversations.  \n\nCould you please tell me the math problem you'd like help with? I'm ready! 😊  \n\n"